# Applied Data Science Capstone Course by Coursera (IBM)¶


## Finding Pub Clusters in Mumbai (aka Bombay)
### Final Project Python Notebook

- Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new Pub

#### Import Library

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


#### Fetch dataset from Wiki

In [6]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [7]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [8]:
# create a list to store neighborhood data
neighborhoodList = []

In [9]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [60]:
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup
5,Borivali
6,Charkop
7,Chembur
8,Dahisar
9,Devipada


In [11]:
# print the number of rows of the dataframe
kl_df.shape

(42, 1)

#### Find Coordinates

In [13]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [14]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [19]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [21]:
df_coords.shape

(42, 2)

In [23]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [24]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(42, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062180,72.902410
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [25]:
# get the coordinates of Mumbai
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


#### Map of Mumbai

In [59]:
# create map of Mumbai using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

#### Foursquare API details

In [27]:
# Foursquare Credentials
CLIENT_ID = 'WYW3VGFA2DJ1QG05W1R2P55N13O25OPLDRU5IGYGRAC32IY4'
CLIENT_SECRET = 'BEDUOMVC3LEVBDYSDOKNQYK5UL3O54CRWNVHXX3X3CSYOZ0C' 
VERSION = '20200409' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: WYW3VGFA2DJ1QG05W1R2P55N13O25OPLDRU5IGYGRAC32IY4
CLIENT_SECRET:BEDUOMVC3LEVBDYSDOKNQYK5UL3O54CRWNVHXX3X3CSYOZ0C


#### Let's get the top 100 venues that are within a radius of 2000 meters.

In [28]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [29]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2730, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
3,Andheri,19.118459,72.841763,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
4,Andheri,19.118459,72.841763,Temptations,19.113767,72.841337,Ice Cream Shop


In [30]:
venues_df.groupby(["Neighborhood"]).count()


,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,17,17,17,17,17,17
Baiganwadi,7,7,7,7,7,7
Bandra,100,100,100,100,100,100
Bhandup,26,26,26,26,26,26
Borivali,100,100,100,100,100,100
Charkop,54,54,54,54,54,54
Chembur,97,97,97,97,97,97
Dahisar,60,60,60,60,60,60


#### Unique categories

In [31]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))


There are 200 uniques categories.


In [35]:
# print out the list of categories
venues_df['VenueCategory'].unique()

array(['Bakery', 'Indian Restaurant', 'Ice Cream Shop',
       'Falafel Restaurant', 'Fast Food Restaurant', 'Pub',
       'Sandwich Place', 'Juice Bar', 'Pizza Place', 'Seafood Restaurant',
       'Multiplex', 'Coffee Shop', 'Breakfast Spot', 'Snack Place',
       'Maharashtrian Restaurant', 'Café', 'American Restaurant',
       'Chinese Restaurant', 'Cocktail Bar', 'BBQ Joint', 'Bar', 'Diner',
       "Women's Store", 'Lounge', 'Department Store',
       'Gym / Fitness Center', 'Residential Building (Apartment / Condo)',
       'Spa', 'Asian Restaurant', 'Electronics Store', 'Smoke Shop',
       'Food Truck', 'Vegetarian / Vegan Restaurant', 'Liquor Store',
       'Fish Market', 'Athletics & Sports', 'Martial Arts Dojo',
       'Tea Room', 'Dessert Shop', 'Clothing Store', 'Food', 'Plaza',
       'Bus Station', 'Mughlai Restaurant', 'Platform',
       'Paper / Office Supplies Store', 'Food & Drink Shop',
       'Music Venue', 'Tunnel', 'Fried Chicken Joint', 'Gym', 'Park',
       'Spo

In [61]:
# check if the results contain "Pub"
"Pub" in venues_df['VenueCategory'].unique()

True

#### Analyze Each Neighborhood

In [36]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2730, 201)


,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Campground,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Track Stadium,Trail,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store
0,Andheri,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [37]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(42, 201)


,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Campground,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,English Restaurant,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gujarati Restaurant,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Track Stadium,Trail,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store
0,Andheri,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.020000,0.010000,0.010000,0.00,0.020000,0.000000,0.050000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.00,0.00,0.060000,0.010000,0.020000,0.020000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.010000,0.00,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.020000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.050000,0.000000,0.0,0.00000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.170000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.01,0.020000,0.010000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.00,0

In [39]:
kl_pub = kl_grouped[["Neighborhoods","Pub"]]
kl_pub

,Neighborhoods,Pub
0,Andheri,0.050000
1,Anushakti Nagar,0.058824
2,Baiganwadi,0.000000
3,Bandra,0.010000
4,Bhandup,0.038462
5,Borivali,0.000000
6,Charkop,0.018519
7,Chembur,0.000000
8,Dahisar,0.000000
9,Devipada,0.000000


#### Run k-means to cluster the neighborhoods in Mumbai into 5 clusters.

In [40]:
# set number of clusters
kclusters = 5

kl_clustering = kl_pub.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 0, 3, 2, 0, 3, 0, 0, 0])

In [47]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_pub.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [48]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged

,Neighborhood,Pub,Cluster Labels
0,Andheri,0.050000,4
1,Anushakti Nagar,0.058824,4
2,Baiganwadi,0.000000,0
3,Bandra,0.010000,3
4,Bhandup,0.038462,2
5,Borivali,0.000000,0
6,Charkop,0.018519,3
7,Chembur,0.000000,0
8,Dahisar,0.000000,0
9,Devipada,0.000000,0


In [49]:
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(42, 5)


,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
0,Andheri,0.050000,4,19.118459,72.841763
1,Anushakti Nagar,0.058824,4,19.042830,72.927340
2,Baiganwadi,0.000000,0,19.062940,72.926630
3,Bandra,0.010000,3,19.054370,72.840170
4,Bhandup,0.038462,2,19.145560,72.948560


In [50]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(42, 5)


,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
31,Shil Phata,0.000000,0,19.146580,73.040050
24,"Matharpacady, Mumbai",0.000000,0,18.950694,72.827268
21,Kurla,0.000000,0,19.064980,72.880690
28,Mumbra,0.000000,0,19.167632,73.021408
19,Kanjurmarg,0.000000,0,19.131380,72.935680
29,Pestom sagar,0.000000,0,19.070640,72.902170
17,Kalyan,0.000000,0,18.953937,72.820367
16,Juhu,0.000000,0,19.014920,72.845220
14,Grant Road,0.000000,0,18.959290,72.831080
25,Mira Road,0.000000,0,19.265674,72.870681


In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]


,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
31,Shil Phata,0.0,0,19.146580,73.040050
24,"Matharpacady, Mumbai",0.0,0,18.950694,72.827268
21,Kurla,0.0,0,19.064980,72.880690
28,Mumbra,0.0,0,19.167632,73.021408
19,Kanjurmarg,0.0,0,19.131380,72.935680
29,Pestom sagar,0.0,0,19.070640,72.902170
17,Kalyan,0.0,0,18.953937,72.820367
16,Juhu,0.0,0,19.014920,72.845220
14,Grant Road,0.0,0,18.959290,72.831080
25,Mira Road,0.0,0,19.265674,72.870681


In [53]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
30,Seven Bungalows,0.090000,1,19.131342,72.816342
23,Mankhurd,0.076923,1,19.048530,72.932220
20,Kausa,0.090000,1,19.127580,72.825390


In [54]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
27,Mulund,0.037975,2,19.17183,72.95565
41,Worli,0.030000,2,19.00744,72.81688
4,Bhandup,0.038462,2,19.14556,72.94856


In [55]:
kl_merged.loc[kl_merged['Cluster Labels'] == 3]


,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
18,Kandivali,0.013699,3,19.211982,72.837573
13,Goregaon,0.010000,3,19.164550,72.849460
12,Ghatkopar,0.012048,3,19.086523,72.909008
6,Charkop,0.018519,3,19.208660,72.826120
3,Bandra,0.010000,3,19.054370,72.840170
37,Vashi,0.010000,3,19.084650,72.904810
38,Vikhroli,0.010000,3,19.111090,72.927810
22,Mahavir Nagar (Kandivali),0.013514,3,19.210940,72.841370


In [56]:
kl_merged.loc[kl_merged['Cluster Labels'] == 4]

,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
40,Western Suburbs (Mumbai),0.049383,4,19.197010,72.827680
15,Jogeshwari,0.055556,4,19.137920,72.849410
33,"Sonapur, Bhandup",0.045455,4,19.163940,72.935440
1,Anushakti Nagar,0.058824,4,19.042830,72.927340
0,Andheri,0.050000,4,19.118459,72.841763


#### Observations

Mumbai is the economic capital of India. The Richest man in Antila to the Poorest man in Dharavi live in this city. Thus having a complete diversity among the social and economic groups, making a perfect scenario of every type of recreational facility to be built.

I took Pub as one of such facility, there used to be a time when richest and most affluent could make their way in. But these days as economic growth of the city is rampant it makes it obvious that Pubs could now cater to more people. Therefore a common business model could be put into place, wherein localities, earning status, lifestyle parameters and other factors can determine opening of such recreational spots. My 5 clusters clearly represents some densely and sparsely populated pubs across the city. Cluster 0 shows a high number of such places and other have less than 10. Therefore venturing in those places will not just give business dividends but also a quality life to people living in there.